![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F05+-+TensorFlow&dt=05Tools+-+Automation.ipynb)

# 05Tools - Automation

# IN ACTIVE DEVELOPMENT

<p align="center" width="100%">
    <img src="../architectures/overview/automation.png" width="100%">
</p>
    
This notebook will focus on some helpful GCP tools for automation. In Vertex AI, Training Jobs and Pipelines are triggered to run with the Vertex AI Client library:

- [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference)
    - [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)
        - [`aiplatform` package](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
            - Training Jobs
                - [`aiplatform.CustomJob.from_local_script()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob_from_local_script)
                    - 05a, Using a Python Script
                - [`aiplatform.CustomJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob)
                    - with [worker_pool_specs](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.WorkerPoolSpec) using `python_package_spec`
                    - 05b, Using a Python Source Distribution
                - [`aiplatform.CustomJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob)
                    - with [worker_pool_specs](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.WorkerPoolSpec) using `container_spec`
                    - 05c, Using a Custom Container
                - [`aiplatform.CustomTrainingJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomTrainingJob#google_cloud_aiplatform_CustomTrainingJob)
                    - 05d, Using a Python Script
                - [`aiplatform.CustomPythonPackageTrainingJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomPythonPackageTrainingJob)
                    - 05e, Using a Python Source Distribution
                - [`aiplatform.CustomContainerTrainingJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomContainerTrainingJob#google_cloud_aiplatform_CustomContainerTrainingJob)
                    - 05f, Using a Custom Container
                - Hyperparameter Tuning Job:
                    - [`aiplatform.HyperparameterTuningJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.HyperparameterTuningJob#google_cloud_aiplatform_HyperparameterTuningJob)
                    - In 05g, 05h, and 05i
            - Pipelines
                - [`aiplatform.PipelineJob`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob)
                
This notebook will show using a Python Function to trigger jobs with the SDK summarized above.  The function will be run in a Cloud Function.  Functions are triggered, in this case triggering will be done with a Cloud Pub/Sub Topic.  Message will be sent to the Pub/Sub topic manually and on a schedule by using Cloud Scheduler.

**Prerequisites:**
- 05f

**Resources:**
- Documentation
    - [Schedule pipeline exectuion with Cloud Scheduler](https://cloud.google.com/vertex-ai/docs/pipelines/schedule-cloud-scheduler#create_a_cloud_function_with_http_trigger)
    - [Trigger a pipeline run with Pub/Sub](https://cloud.google.com/vertex-ai/docs/pipelines/trigger-pubsub)

**Conceptual Flow & Workflow**
                


---
## Setup

### Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (Artifact Registry and Cloud Build)
- Artifact Registry
    - to create repositories for Python packages and Docker containers
- Cloud Pub/Sub
- Cloud Functions
- Cloud Scheduler

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [1]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [2]:
try:
    import google.cloud.pubsub
except ImportError:
    print('You need to pip install google-cloud-pubsub')
    !pip install google-cloud-pubsub -q

In [3]:
try:
    import google.cloud.functions
except ImportError:
    print('You need to pip install google-cloud-functions')
    !pip install google-cloud-functions -q

In [4]:
try:
    import google.cloud.scheduler
except ImportError:
    print('You need to pip install google-cloud-scheduler')
    !pip install google-cloud-scheduler -q

### Environment

inputs:

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [6]:
REGION = 'us-central1'
EXPERIMENT = 'automation'
SERIES = '05'

packages:

In [7]:
from google.cloud import aiplatform
from datetime import datetime
import pkg_resources
from IPython.display import Markdown as md
from google.cloud import service_usage_v1
from google.cloud import artifactregistry_v1
from google.cloud import pubsub_v1
from google.cloud import functions_v1
from google.cloud import scheduler_v1
#from google.cloud
#from google.cloud
from google.cloud import storage
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from google.protobuf.duration_pb2 import Duration
import json
import os
import shutil
import zipfile
import numpy as np
import pandas as pd

clients:

In [8]:
gcs = storage.Client()
su_client = service_usage_v1.ServiceUsageClient()
pubsub_pubclient = pubsub_v1.PublisherClient() 
functions_client = functions_v1.CloudFunctionsServiceClient()
scheduler_client = scheduler_v1.CloudSchedulerClient()

parameters:

In [9]:
DIR = f"temp/{EXPERIMENT}"

In [10]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

List the service accounts current roles:

In [11]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [12]:
!rm -rf {DIR}
!mkdir -p {DIR}

### Enable APIs

Using Cloud Functions, Cloud Pub/Sub and Cloud Scheduler requires enabling these APIs for the Google Cloud Project.

Options for enabeling these.  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Cloud Build and Enable
     - Search for Artifact Registry and Enable
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### Cloud Pub/Sub

In [13]:
pubsub = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/pubsub.googleapis.com'
    )
).state.name


if pubsub == 'DISABLED':
    print(f'Cloud Pub/Sub is currently {pubsub} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/pubsub.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Pub/Sub is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Pub/Sub already enabled for project: {PROJECT_ID}')

Cloud Pub/Sub already enabled for project: statmike-mlops-349915


#### Cloud Functions

In [14]:
cloudfunctions = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/cloudfunctions.googleapis.com'
    )
).state.name


if cloudfunctions == 'DISABLED':
    print(f'Cloud Functions is currently {cloudfunctions} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/cloudfunctions.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Functions is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Functions already enabled for project: {PROJECT_ID}')

Cloud Functions already enabled for project: statmike-mlops-349915


#### Cloud Scheduler

In [15]:
cloudscheduler = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/cloudscheduler.googleapis.com'
    )
).state.name


if cloudscheduler == 'DISABLED':
    print(f'Cloud Scheduler is currently {cloudscheduler} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/cloudscheduler.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Scheduler is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Scheduler already enabled for project: {PROJECT_ID}')

Cloud Scheduler already enabled for project: statmike-mlops-349915


---
## Pub/Sub

The main concepts:
- Topic - a feed of messages
     - Publish - send a new message to a topic
     - Subscription - receive messages that arrive on topic
          - Push - the subscriber has new messages pushed to it
          - Pull - the subscriber request new messages by pulling them
          
In this example, a topic will be set up for training the model in the local `EXPERIMENT`.  Publishing a new message to this topic will trigger a training run initiated by the Cloud Function (setup below).  The Cloud Funtion will have a push subscription to the topic.

In [16]:
PUBSUB_TOPIC = f'train-{SERIES}-{EXPERIMENT}'

In [17]:
for topic in pubsub_pubclient.list_topics(project = f'projects/{PROJECT_ID}'):
    if topic.name.endswith(PUBSUB_TOPIC):
        break
    else: topic = ''

In [69]:
if topic:
    print(topic)
else:
    topic = pubsub_pubclient.create_topic(
        name = pubsub_pubclient.topic_path(PROJECT_ID, PUBSUB_TOPIC)
    )
    print(topic)

name: "projects/statmike-mlops-349915/topics/train-05-automation"



---
## Cloud Functions

### Create Files for Function

In [20]:
if os.path.exists(f'{DIR}/function'): shutil.rmtree(f'{DIR}/function')
os.makedirs(f'{DIR}/function')

In [21]:
%%writefile {DIR}/function/main.py

# packages
import base64
import json
from google.cloud import aiplatform
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

# function that runs the code from the 05f notebook with inputs
def train(event, context):
    #Triggered from a message on a Cloud Pub/Sub topic.
    
    # decode the data input from the event dictionary and convert to Python dictionary
    function_input = json.loads(base64.b64decode(event['data']).decode('utf-8'))
    print(function_input)
    
    # INPUTS
    EPOCHS = function_input['EPOCHS']
    BATCH_SIZE = function_input['BATCH_SIZE']
    VAR_TARGET = function_input['VAR_TARGET']
    VAR_OMIT = function_input['VAR_OMIT']
    PROJECT_ID = function_input['PROJECT_ID']
    BQ_PROJECT = function_input['BQ_PROJECT']
    BQ_DATASET = function_input['BQ_DATASET']
    BQ_TABLE = function_input['BQ_TABLE']
    REGION = function_input['REGION']
    EXPERIMENT = function_input['EXPERIMENT']
    SERIES = function_input['SERIES']
    EXPERIMENT_NAME = function_input['EXPERIMENT_NAME']
    RUN_NAME = f'run-{TIMESTAMP}'
    TRAIN_COMPUTE = function_input['TRAIN_COMPUTE']
    DEPLOY_IMAGE = function_input['DEPLOY_IMAGE']
    URI = function_input['URI']
    REPOSITORY = function_input['REPOSITORY']
    SERVICE_ACCOUNT = function_input['SERVICE_ACCOUNT']
    
    # clients
    aiplatform.init(project = PROJECT_ID, location = REGION)
    
    # Get Vertex AI Experiment Tensorboard Instance Name
    tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
    if tb:
        tb = tb[0]
    else: 
        tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})
    
    # Setup Vertex AI Experiment
    aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)
    
    # Setup Training Job
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--batch_size=" + str(BATCH_SIZE),
        "--var_target=" + VAR_TARGET,
        "--var_omit=" + VAR_OMIT,
        "--project_id=" + PROJECT_ID,
        "--bq_project=" + BQ_PROJECT,
        "--bq_dataset=" + BQ_DATASET,
        "--bq_table=" + BQ_TABLE,
        "--region=" + REGION,
        "--experiment=" + EXPERIMENT,
        "--series=" + SERIES,
        "--experiment_name=" + EXPERIMENT_NAME,
        "--run_name=" + RUN_NAME
    ]

    trainingJob = aiplatform.CustomContainerTrainingJob(
        display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
        container_uri = f"{REPOSITORY}/{EXPERIMENT}_trainer",
        model_serving_container_image_uri = DEPLOY_IMAGE,
        staging_bucket = f"{URI}/models/{TIMESTAMP}",
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
    ) 
    
    # Run Training Job AND Upload The Model
    modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

    upload_model = True
    if modelmatch:
        print("Model Already in Registry:")
        if RUN_NAME in modelmatch[0].version_aliases:
            print("This version already loaded, no action taken.")
            upload_model = False
            model = aiplatform.Model(model_name = modelmatch[0].resource_name)
        else:
            print('Loading model as new default version.')
            parent_model = modelmatch[0].resource_name
    else:
        print('This is a new model, creating in model registry')
        parent_model = ''

    if upload_model:
        model = trainingJob.run(
            model_display_name = f'{SERIES}_{EXPERIMENT}',
            model_labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'},
            model_id = f'model_{SERIES}_{EXPERIMENT}',
            parent_model = parent_model,
            is_default_version = True,
            model_version_aliases = [RUN_NAME],
            model_version_description = RUN_NAME,
            base_output_dir = f"{URI}/models/{TIMESTAMP}",
            service_account = SERVICE_ACCOUNT,
            args = CMDARGS,
            replica_count = 1,
            machine_type = TRAIN_COMPUTE,
            accelerator_count = 0,
            tensorboard = tb.resource_name
        )
    
    # THIS FUNCTION WILL TIMEOUT BEFORE IT CAN RUN THE FOLLOWING
    # CONSIDER A SEPERATE FUNCTION THAT IS TRIGGER BY A MODEL REGISTY UPDATE
    
    # Vertex AI Experiment Update and Review
    expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)
    expRun.log_params({
        'model.uri': model.uri,
        'model.display_name': model.display_name,
        'model.name': model.name,
        'model.resource_name': model.resource_name,
        'model.version_id': model.version_id,
        'model.versioned_resource_name': model.versioned_resource_name,
        'customJobs.display_name': customJob.display_name,
        'customJobs.resource_name': customJob.resource_name,
        'customJobs.link': job_link,
        'customJobs.tensorboard': board_link
    })
    expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Writing temp/automation/function/main.py


In [22]:
%%writefile {DIR}/function/requirements.txt
pandas
google-cloud-aiplatform

Writing temp/automation/function/requirements.txt


### Store Files in Cloud Storage

Copy from local folder (`DIR/function`) to GCS at the path `SERIES/EXPERIMENT/function`:

In [23]:
!ls {DIR}/function

main.py  requirements.txt


In [24]:
!cd {DIR}/function && tar czvf function.tar.gz main.py requirements.txt

main.py
requirements.txt


In [25]:
with zipfile.ZipFile(f'{DIR}/function/function.zip', mode = 'w') as archive:
    archive.write(f'{DIR}/function/main.py', 'main.py')
    archive.write(f'{DIR}/function/requirements.txt', 'requirements.txt')

In [26]:
!ls {DIR}/function

function.tar.gz  function.zip  main.py	requirements.txt


In [27]:
with zipfile.ZipFile(f'{DIR}/function/function.zip', mode = 'r') as zip:
    zip.printdir()

File Name                                             Modified             Size
main.py                                        2022-11-01 22:39:20         5206
requirements.txt                               2022-11-01 22:39:22           31


In [28]:
bucket = gcs.lookup_bucket(PROJECT_ID)
SOURCEPATH = f'{SERIES}/{EXPERIMENT}/function'

In [29]:
blob = bucket.blob(f'{SOURCEPATH}/function.zip')
blob.upload_from_filename(f'{DIR}/function/function.zip')

In [30]:
list(bucket.list_blobs(prefix = f'{SOURCEPATH}'))

[<Blob: statmike-mlops-349915, 05/automation/function/function.zip, 1667342376889481>]

In [31]:
print(f"View the bucket directly here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SOURCEPATH};tab=objects&project={PROJECT_ID}")

View the bucket directly here:
https://console.cloud.google.com/storage/browser/statmike-mlops-349915/05/automation/function;tab=objects&project=statmike-mlops-349915


### Create (or Update) Cloud Function

In [ ]:
function_name = f'train-{SERIES}-{EXPERIMENT}'

In [45]:
function = ''
for function in functions_client.list_functions(request = functions_v1.ListFunctionsRequest(parent = f'projects/{PROJECT_ID}/locations/{REGION}')):
    if function.name.endswith(function_name):
        break
    else: function = ''

In [46]:
functionDef = functions_v1.CloudFunction()
functionDef.name = f'projects/{PROJECT_ID}/locations/{REGION}/functions/{function_name}'
functionDef.source_archive_url = f"gs://{PROJECT_ID}/{SOURCEPATH}/function.zip"
functionDef.event_trigger = functions_v1.EventTrigger()
functionDef.event_trigger.event_type = 'providers/cloud.pubsub/eventTypes/topic.publish'
functionDef.event_trigger.resource = topic.name
functionDef.runtime = 'python310'
functionDef.entry_point = 'train'
functionDef.timeout = Duration(seconds = 120)
functionDef.service_account_email = SERVICE_ACCOUNT

In [47]:
functionDef

name: "projects/statmike-mlops-349915/locations/us-central1/functions/train-05-automation"
source_archive_url: "gs://statmike-mlops-349915/05/automation/function/function.zip"
event_trigger {
  event_type: "providers/cloud.pubsub/eventTypes/topic.publish"
  resource: "projects/statmike-mlops-349915/topics/train-05-automation"
}
entry_point: "train"
timeout {
  seconds: 120
}
service_account_email: "1026793852137-compute@developer.gserviceaccount.com"
runtime: "python310"

In [48]:
function

''

In [49]:
if function:
    request = functions_v1.UpdateFunctionRequest(
        function = functionDef
    )
    operation = functions_client.update_function(request = request)
else:
    request = functions_v1.CreateFunctionRequest(
        location = f"projects/{PROJECT_ID}/locations/{REGION}",
        function = functionDef
    )
    operation = functions_client.create_function(request = request)

In [50]:
response = operation.result()
print(response)

name: "projects/statmike-mlops-349915/locations/us-central1/functions/train-05-automation"
source_archive_url: "gs://statmike-mlops-349915/05/automation/function/function.zip"
event_trigger {
  event_type: "providers/cloud.pubsub/eventTypes/topic.publish"
  resource: "projects/statmike-mlops-349915/topics/train-05-automation"
  service: "pubsub.googleapis.com"
  failure_policy {
  }
}
status: ACTIVE
entry_point: "train"
timeout {
  seconds: 120
}
available_memory_mb: 256
service_account_email: "1026793852137-compute@developer.gserviceaccount.com"
update_time {
  seconds: 1667342811
  nanos: 389000000
}
version_id: 1
runtime: "python310"
ingress_settings: ALLOW_ALL
build_id: "cf7df652-97b3-485a-b2a2-b759b1d6316c"
build_name: "projects/1026793852137/locations/us-central1/builds/cf7df652-97b3-485a-b2a2-b759b1d6316c"
docker_registry: CONTAINER_REGISTRY



In [51]:
print(f'Review the Cloud Function in the console here:\nhttps://console.cloud.google.com/functions/list?env=gen1&project={PROJECT_ID}')

Review the Cloud Function in the console here:
https://console.cloud.google.com/functions/list?env=gen1&project=statmike-mlops-349915


---
## Manual Run of Cloud Function

Publish a message to the Pub/Sub topic that will cause the Cloud Function to initiate training.  The code below could be anywhere you want to trigger training!

The function will receive the message as `event` in the format:
```
{
    '@type': 'type.googleapis.com/google.pubsub.v1.PubsubMessage',
    'attributes': {'key' : 'value', ...},
    'data': <base64 encoded string>
}
```

To handle the `event` and retrieve the inputs of the message three things need to happen:
1. reference the 'data' value as `event['data']`
2. decode the 'data' value with `base64.b64decode(<1>).decode('utf-8')`
3. convert the decoded string into a Python dictionary with `json.loads(<2>)`

This looks like:
```
funtion_inputs = json.loads(base64.b64decode(event['data']).decode('utf-8'))
```

In [52]:
EXPERIMENT_NAME = f'experiment-{SERIES}-05f-tf-classification-dnn'
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/05f"
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PROJECT_ID}"

function_input = {
    'EPOCHS': 10,
    'BATCH_SIZE': 100,
    'VAR_TARGET': 'Class',
    'VAR_OMIT': 'transaction_id',
    'PROJECT_ID': PROJECT_ID,
    'BQ_PROJECT': PROJECT_ID,
    'BQ_DATASET': 'fraud',
    'BQ_TABLE': 'fraud_prepped',
    'REGION': REGION,
    'EXPERIMENT': '05f',
    'SERIES': SERIES,
    'EXPERIMENT_NAME': EXPERIMENT_NAME,
    'TRAIN_COMPUTE': 'n1-standard-4',
    'DEPLOY_IMAGE': 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest',
    'URI': URI,
    'REPOSITORY': REPOSITORY,
    'SERVICE_ACCOUNT': SERVICE_ACCOUNT
}

In [53]:
message = json.dumps(function_input)
message = message.encode('utf-8')

In [54]:
future = pubsub_pubclient.publish(topic.name, message, trigger = 'manual')

In [55]:
future.result()

'6140257232683221'

---
## Scheduled Run with Cloud Scheduler

Use Cloud Scheduler to publish a message to the topic at any defined interval which will cause the Cloud Function to initiate training.

Resources:
- List of Time zones - [TZ Database Names](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)
- Job Frequency - [unix-cron format guide](https://man7.org/linux/man-pages/man5/crontab.5.html)
    - minute hour day_of_month month day_of_week
    - 0 23 * * tue = 11PM every Tuesday



In [70]:
schedule_name = f'train-{SERIES}-{EXPERIMENT}'

In [71]:
schedule = ''
for schedule in scheduler_client.list_jobs(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if schedule.name.endswith(schedule_name):
        break
    else: schedule = ''

In [72]:
if schedule:
    print(schedule)
else:
    request = scheduler_v1.CreateJobRequest(
        parent = f'projects/{PROJECT_ID}/locations/{REGION}',
        job = scheduler_v1.Job(
            name = f'projects/{PROJECT_ID}/locations/{REGION}/jobs/{schedule_name}',
            pubsub_target = scheduler_v1.PubsubTarget(
                topic_name = topic.name,
                data = message,
                attributes = {'trigger': 'scheduled'}
            ),
            schedule = '0 23 * * tue',
            time_zone = 'America/New_York'
        )
    )
    schedule = scheduler_client.create_job(request = request)
    print(schedule)

name: "projects/statmike-mlops-349915/locations/us-central1/jobs/train-05-automation"
pubsub_target {
  topic_name: "projects/statmike-mlops-349915/topics/train-05-automation"
  data: "{\"EPOCHS\": 10, \"BATCH_SIZE\": 100, \"VAR_TARGET\": \"Class\", \"VAR_OMIT\": \"transaction_id\", \"PROJECT_ID\": \"statmike-mlops-349915\", \"BQ_PROJECT\": \"statmike-mlops-349915\", \"BQ_DATASET\": \"fraud\", \"BQ_TABLE\": \"fraud_prepped\", \"REGION\": \"us-central1\", \"EXPERIMENT\": \"05f\", \"SERIES\": \"05\", \"EXPERIMENT_NAME\": \"experiment-05-05f-tf-classification-dnn\", \"TRAIN_COMPUTE\": \"n1-standard-4\", \"DEPLOY_IMAGE\": \"us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest\", \"URI\": \"gs://statmike-mlops-349915/05/05f\", \"REPOSITORY\": \"us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915\", \"SERVICE_ACCOUNT\": \"1026793852137-compute@developer.gserviceaccount.com\"}"
  attributes {
    key: "trigger"
    value: "scheduled"
  }
}
user_update_time {
  seconds:

In [73]:
print(f'Review the Cloud Scheduler in the console here:\nhttps://console.cloud.google.com/cloudscheduler?&project={PROJECT_ID}')

Review the Cloud Scheduler in the console here:
https://console.cloud.google.com/cloudscheduler?&project=statmike-mlops-349915
